# Data retrival

In [6]:
import myfunctions as f
print f.today()

20180201


In [7]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 
# setup elastic search
f.initialise_es(1)
foodpanda, deliveroo, wte, websites = f.delivery_para()

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712
{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}



# To retrive food data

In [3]:
doc_type = "food"

In [6]:
%%time
website = websites[0]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 2h 5min 52s, sys: 8min 52s, total: 2h 14min 45s
Wall time: 2h 15min 15s


In [7]:
%%time
website = websites[1]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 1h 4min 58s, sys: 2min 39s, total: 1h 7min 37s
Wall time: 1h 8min 5s


In [8]:
%%time
website = websites[2]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 11min 8s, sys: 16.6 s, total: 11min 24s
Wall time: 11min 29s


## Data: keep all index and dates

In [9]:
### fomulate location
def fomulate_loc(w):
    """formulate storage location according to es data"""
    w['all_rec_food']["loc"] = w['all_rec_food']["_index"] + "/" + \
    w["food"] + "/" + \
    w['all_rec_food']["_id"]

def fomulate_timestamp(w):
    if w["_index"] == 'what_to_eat':
        w['all_rec_food']["timestamp"] = w['all_rec_food']['_source.startTimestampGMT']
    else:
        d = f.retrive_from_es(w, "crawling_cycle")
        cycle_time = d.set_index(w['cycle_id']).to_dict()[w['cycleStart']]
        # remove na for food names
        w['all_rec_food'] = w['all_rec_food'][~w['all_rec_food'][w["food_name"]].isnull()]
        # assign time according to food cycle
        w['all_rec_food']["timestamp"] = w['all_rec_food'][w['food_cycle']].apply(lambda s: cycle_time[s])

def formated(w):
    cols = [w['food_name'], w['vendor_name'], "timestamp", "loc"]
    temp = w['all_rec_food'][cols]
    temp.columns = ["food_name", "vendor_name","timestamp", "loc" ]
    temp["locs"] = temp["timestamp"]+ "****" + temp["loc"]
    temp["locs"] = temp["locs"].apply(lambda s: tuple(s.split("****")))
    temp = temp[["food_name", "vendor_name", "locs" ]]
    w["formated"] = temp

In [10]:
for w in websites:
    fomulate_loc(w)     
    fomulate_timestamp(w) 
    formated(w)
    print "%s:: number of records: %d"%(w["_index"],
                                        w['all_rec_food'].shape[0])

foodpanda:: number of records: 1764391
deliveroo:: number of records: 1136336
what_to_eat:: number of records: 169196


# To save food data

In [23]:
for w in websites:
    file_name = "__".join([w["_index"], "food", today]) + ".p"
    f.save_file(w['all_rec_food'], file_name)

saved: foodpanda__food__20180102.p
saved: deliveroo__food__20180102.p
saved: what_to_eat__food__20180102.p


In [ ]:
import pandas as pd

file_name = "__".join(["all_food_vendor", f.today()]) + ".p"
df1 = pd.concat([w["formated"] for w in websites]) 
df = df1.groupby(["food_name", "vendor_name"])["locs"].apply(list).to_frame().reset_index()

In [21]:
print "all:: number of records: %d"%(df.shape[0])
f.save_file(df, file_name)

all:: number of records: 445939
saved: all_food_vendor__20180102.p


# Clean the food items: get clean name

In [ ]:
file_name = "all_food_vendor__20180102.p"

In [3]:
drink_list = f.retrive_drinks()
df = f.retrive_file(file_name)
df["clean_name"] = df["food_name"].apply(lambda s: f.clean_name_v4(s))

df1 = df[~df['clean_name'].isin(drink_list)]
print ("Got %d non-drink/dessert food-vendor records" % df1.shape[0])
print ("Got %d unique non-drink/dessert food " % 
       df1["clean_name"].nunique())

file_name = "__".join(["all_food_vendor","clean", f.today()]) + ".p"
f.save_file(df1, file_name)

number of drinks in drink list: 22452
retrived: all_food_vendor__20180102.p
Got 333575 non-drink/dessert food-vendor records
Got 98094 unique non-drink/dessert food 
saved: all_food_vendor__clean__20180118.p


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def lemma_name(s):
    return " ".join([token.lemma_ for token in nlp(unicode(s, "utf-8"))])

df["lemma_name"] = df["clean_name"].apply(lambda s: lemma_name(s))
df["lemma_name_2"] = df["lemma_name"].apply(lambda s: lemma_name(str(s)))

There is very little difference generated by doing lemmatization twice. 

    positive case for lemmatization x 2: dumplings -> dumpling; dunpling -> dumpl
    negative case : shreded -> shred -> shr 

Thus, we only consider doing it once.

In [ ]:
file_name = "__".join(["all_food_vendor","lemma", f.today()]) + ".p"
f.save_file(df, file_name)

    saved: all_food_vendor__lemma__20180122.p